# **RentaBot: The Apartment Information Bot**


#### **RentaBot** is an intelligent, interactive chatbot designed to help users find the perfect apartment or shared house within a specific apartment complex. Acting as a virtual assistant, it guides potential tenants by answering their questions and providing detailed information about available units, amenities, and rental options. This bot not only helps users find suitable apartments but also assists in scheduling tours and collecting contact information for follow-up. It ensures a smooth and efficient apartment-hunting experience.


### Importing Required Libraries

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load the local dotenv file where the OpenAI API key is stored
_=load_dotenv(find_dotenv())

# Set the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

### Helper Function for Chatting with the Model

In [2]:
# Helper function to interact with the model
def get_completion(messages, model="gpt-3.5-turbo", temperature=0):
    """
    Sends a list of messages to the OpenAI API and retrieves the assistant's response.

    Parameters:
    - messages (list): A list of dictionaries representing the conversation history.
    - model (str): The name of the model to use (default is "gpt-3.5-turbo").
    - temperature (float): Controls the randomness of the output (default is 0 for deterministic responses).

    Returns:
    - str: The assistant's response.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

#### In order to make coversational chat bot we need to stack all prompt and compelitions (answers form the model) to gether. We can can do so using the function below

### Function to Collect Messages for Conversational Chatbot

In [5]:
# Function to manage the conversation by stacking user prompts and bot responses
def collect_messages(_):
     """
    Handles user input, sends it to the model, and updates the conversation panel.

    Parameters:
    - _ (event): The event object from the Panel widget (not used here).
    """
    prompt = input_text.value_input
    input_text.value = '' # Clear the input field after submission

    # Add the user's message to the conversation context
    context.append({'role':'user', 'content':f"{prompt}"})

    # Get the assistant's response using the helper function
    response = get_completion(context) 

    # Add the assistant's response to the conversation context
    context.append({'role':'assistant', 'content':f"{response}"})

    # Update the display panel with user input and assistant's response
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
 
    return pn.Column(*panels)


### Using Panel to Create a GUI

In [ ]:
import panel as pn  # GUI

# Initialize the Panel extension for the GUI
pn.extension()

# List to collect conversation panels
panels = [] 

# Initial conversation context with system prompt defining the bot's behavior
context = [ {'role':'system', 'content':"""
You are RentaBot, a virtual assistant that helps users find the perfect apartment or shared house in our apartment complex.
You have detailed information about various units, including the number of bedrooms, bathrooms, rent, rental periods,
furnishing options, amenities, pet-friendliness, floor level, and availability dates.

When users ask about available units, ask follow-up questions step by step to gather their preferences, such as:
1. Number of bedrooms (e.g., 1-bedroom, 2-bedroom, or shared room)
2. Rental period (6 months or 12 months)
3. Whether they need a furnished or unfurnished unit
4. Maximum budget
5. Desired amenities (e.g., parking, gym, pool, garden, laundry)
6. Pet-friendliness
7. Preferred floor level (1, 2, or 3)
8. Move-in date

Once you gather enough details, suggest suitable units. If no units match the exact criteria, suggest the closest available options.
Always respond in a friendly and helpful tone. If the user was interested to what you suggested ask them if they want to scedule a time to visit the unit 
and take their email address, and tell them we will contact them first available time for schedule a tour.

Here’s the list of available units:

1. **Unit A101**:
   - Type: Apartment
   - 2 Bedrooms, 1 Bathroom
   - Rent: $1500/month
   - Rental Periods: 6 months, 12 months
   - Floor: 1
   - Furnished: No
   - Amenities: Gym, Pool, Parking
   - Pet-friendly: Yes
   - Available From: March 1, 2025

2. **Unit B202**:
   - Type: Apartment
   - 1 Bedroom, 1 Bathroom
   - Rent: $1200/month
   - Rental Periods: 6 months, 12 months
   - Floor: 2
   - Furnished: Yes
   - Amenities: Parking
   - Pet-friendly: No
   - Available From: February 15, 2025

3. **Shared House C101**:
   - Type: Shared House
   - 4 Bedrooms (Shared), 2 Bathrooms (Shared)
   - Rent: $600/month per room
   - Rental Periods: 6 months, 12 months
   - Floor: 1
   - Furnished: Yes
   - Amenities: Kitchen, Laundry, Garden
   - Pet-friendly: No
   - Available From: January 20, 2025

4. **Shared House C102**:
   - Type: Shared House
   - 3 Bedrooms (Shared), 1 Bathroom (Shared)
   - Rent: $650/month per room
   - Rental Periods: 6 months, 12 months
   - Floor: 2
   - Furnished: No
   - Amenities: Kitchen, Garden, Parking
   - Pet-friendly: Yes
   - Available From: February 5, 2025

5. **Unit D303**:
   - Type: Apartment
   - 3 Bedrooms, 2 Bathrooms
   - Rent: $1800/month
   - Rental Periods: 6 months, 12 months
   - Floor: 3
   - Furnished: No
   - Amenities: Gym, Pool, Parking
   - Pet-friendly: Yes
   - Available From: March 10, 2025

6. **Unit E201**:
   - Type: Apartment
   - Studio, 1 Bathroom
   - Rent: $1000/month
   - Rental Periods: 6 months, 12 months
   - Floor: 2
   - Furnished: Yes
   - Amenities: Parking, Laundry
   - Pet-friendly: No
   - Available From: January 30, 2025

7. **Unit F102**:
   - Type: Apartment
   - 2 Bedrooms, 2 Bathrooms
   - Rent: $1600/month
   - Rental Periods: 6 months, 12 months
   - Floor: 1
   - Furnished: Yes
   - Amenities: Gym, Pool, Parking
   - Pet-friendly: Yes
   - Available From: February 25, 2025

8. **Unit G301**:
   - Type: Apartment
   - 1 Bedroom, 1 Bathroom
   - Rent: $1100/month
   - Rental Periods: 12 months
   - Floor: 3
   - Furnished: No
   - Amenities: Parking, Garden
   - Pet-friendly: No
   - Available From: March 5, 2025

9. **Shared House H201**:
   - Type: Shared House
   - 5 Bedrooms (Shared), 3 Bathrooms (Shared)
   - Rent: $550/month per room
   - Rental Periods: 12 months
   - Floor: 2
   - Furnished: Yes
   - Amenities: Kitchen, Garden, Laundry
   - Pet-friendly: Yes
   - Available From: February 1, 2025

10. **Unit I101**:
    - Type: Apartment
    - 2 Bedrooms, 1 Bathroom
    - Rent: $1400/month
    - Rental Periods: 6 months, 12 months
    - Floor: 1
    - Furnished: No
    - Amenities: Gym, Pool
    - Pet-friendly: No
    - Available From: March 15, 2025

"""} ]  # accumulate messages

### Creating the Dashboard

In [ ]:
# Create input and button widgets for user interaction
input_text = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
submit_button = pn.widgets.Button(name="Chat!")

# Bind the button to the collect_messages function
interactive_conversation = pn.bind(collect_messages, submit_button)

# Create the dashboard layout
dashboard = pn.Column(
    input_text,
    pn.Row(submit_button),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

#dashboard
pn.serve(dashboard, start=True, show=True)

Launching server at http://localhost:59889


### Generating a JSON Summary of the User’s Chat

In [12]:
# Create a copy of the current context to generate a summary
messages =  context.copy()

# Add a system prompt to generate a JSON summary
messages.append(
{'role':'system', 'content':'create a json summary of the previous user chat. Provide the email address and the unite you suggessted to the user'},    
)

# Get the response from the model
response = get_completion(messages, temperature=0)
print(response)

{
    "user_preferences": {
        "unit_type": "Apartment",
        "bedrooms": "Any",
        "rental_period": "12 months",
        "furnished": "Yes",
        "budget": "$1500",
        "amenities": ["Parking"],
        "pet_friendly": "No",
        "floor_level": "3",
        "move_in_date": "Any time"
    },
    "recommended_unit": {
        "unit_name": "Unit D303",
        "type": "Apartment",
        "bedrooms": "3",
        "bathrooms": "2",
        "rent": "$1800/month",
        "rental_period": "12 months",
        "floor": "3",
        "furnished": "No",
        "amenities": ["Gym", "Pool", "Parking"],
        "pet_friendly": "Yes",
        "available_from": "March 10, 2025"
    },
    "user_email": "example@gmail.com"
}
